# Classification with KNN, Trees and Gaussian Naive Bayes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Load and split the data from the Unsupervise Learning Dataset (Lab 5, Dry Bean Dataset):

In [2]:
FFILE = './Dry_Bean_Dataset.xlsx'
if os.path.isfile(FFILE):
    print("File already exists")
    if os.access(FFILE, os.R_OK):
        print ("File is readable")
    else:
        print ("File is not readable, removing it and downloading again")
        !rm FFILE
        !wget "https://raw.github.com/alexdepremia/ML_IADA_UTs/main/Lab5/Dry_Bean_Dataset.xlsx"
else:
    print("Either the file is missing or not readable, download it")
    !wget "https://raw.github.com/alexdepremia/ML_IADA_UTs/main/Lab5/Dry_Bean_Dataset.xlsx"

File already exists
File is readable


In [3]:
# Load the data
data = pd.read_excel('./Dry_Bean_Dataset.xlsx')
data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


Divide features and label. Split the data in train and test set and **after that** normalize them:

In [11]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X = data
Y = X['Class']
X = X.drop('Class',axis = 1)
col = X.columns

x_tr, x_te,y_tr,y_te = train_test_split(X,Y,random_state = 5)

tr_scaler = preprocessing.StandardScaler().fit(x_tr)
x_tr = tr_scaler.transform(x_tr)
x_tr = pd.DataFrame(x_tr, columns=col)
x_tr['Class'] = y_tr.values

te_scaler = preprocessing.StandardScaler().fit(x_te)
x_te = te_scaler.transform(x_te)
x_te = pd.DataFrame(x_te, columns=col)
x_te['Class'] = y_te.values

In [12]:
x_tr.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,-0.365400,-0.403300,-0.376399,-0.313440,-0.217938,0.061476,-0.372884,-0.353457,1.288990,1.113580,0.452555,0.107429,0.211468,0.099552,0.069273,-0.075162,SIRA
1,0.898302,1.078153,1.078760,0.987730,0.367612,0.555347,0.886243,1.098425,1.304088,0.707185,-0.460215,-0.471985,-1.221492,-0.987805,-0.500676,-0.151516,CALI
2,-0.112518,-0.160124,-0.483562,0.569790,-1.473833,-1.943742,-0.115282,-0.022695,-0.012195,0.261752,0.932650,1.698110,-0.859059,0.992126,1.766336,0.663225,SEKER
3,-0.807235,-0.999757,-0.886344,-1.091831,0.044749,0.303202,-0.806653,-1.011765,-0.774045,0.088547,0.393328,-0.144208,1.579435,0.505742,-0.181417,0.531998,DERMASON
4,0.495250,0.880614,0.565857,0.706761,-0.031774,0.236545,0.509578,0.682117,-0.845666,-1.532137,-1.592100,-0.061648,-0.991541,-0.618587,-0.099703,0.741324,BARBUNYA


In [13]:
x_tr['Class'].unique()

array(['SIRA', 'CALI', 'SEKER', 'DERMASON', 'BARBUNYA', 'HOROZ', 'BOMBAY'],
      dtype=object)

In [14]:
data = data.sample(frac=1,random_state=0).reset_index(drop=True) # random shuffle
data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,23937,579.371,224.318894,136.195165,1.647040,0.794587,24243,174.578163,0.710381,0.987378,0.896120,0.778259,0.009371,0.002121,0.605687,0.997590,DERMASON
1,84049,1146.398,395.451249,271.309506,1.457565,0.727530,85440,327.130724,0.788083,0.983720,0.803659,0.827234,0.004705,0.001359,0.684316,0.997435,BARBUNYA
2,41240,798.716,315.689810,167.503739,1.884673,0.847625,41796,229.147112,0.614816,0.986697,0.812351,0.725862,0.007655,0.001311,0.526875,0.992987,HOROZ
3,69221,1034.794,420.185894,212.601149,1.976405,0.862552,70186,296.875251,0.803961,0.986251,0.812344,0.706533,0.006070,0.000933,0.499189,0.986600,CALI
4,30440,637.929,236.863145,164.134081,1.443108,0.720987,30717,196.869022,0.773669,0.990982,0.939961,0.831151,0.007781,0.002291,0.690812,0.996916,DERMASON


In [15]:
data.shape

(13611, 17)

In [16]:
data['Class'].unique()

array(['DERMASON', 'BARBUNYA', 'HOROZ', 'CALI', 'BOMBAY', 'SIRA', 'SEKER'],
      dtype=object)

In [17]:
train_data = data.iloc[:10000,:]
test_data = data.iloc[10000:,:]

In [18]:
print(train_data.shape)
print(test_data.shape)

(10000, 17)
(3611, 17)


In [19]:
# normalize train and test dataset
from sklearn import preprocessing

# Extract the class labels from the training dataset
label_train = train_data['Class']
# Remove the class labels from the training dataset
train_data = train_data.drop('Class', axis=1)
# Save the column names for later use
columns_name = train_data.columns

# Initialize a StandardScaler and fit it to the training data
train_scaler = preprocessing.StandardScaler().fit(train_data)
# Transform the training data using the scaler
train_data = train_scaler.transform(train_data)
# Create a DataFrame with the scaled training data and restore column names
train_data = pd.DataFrame(train_data, columns=columns_name)
# Add the class labels back to the scaled training dataset
train_data['Class'] = label_train

# Extract the class labels from the test dataset
label_test = test_data['Class']
# Remove the class labels from the test dataset
test_data = test_data.drop('Class', axis=1)
# Initialize a StandardScaler and fit it to the test data
test_scaler = preprocessing.StandardScaler().fit(test_data)
# Transform the test data using the scaler
test_data = test_scaler.transform(test_data)
# Create a DataFrame with the scaled test data and restore column names
test_data = pd.DataFrame(test_data, columns=columns_name)
# Add the class labels back to the scaled test dataset
test_data['Class'] = label_test.values # .values added to prevent nans to appear

In [20]:
train_data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,-0.997081,-1.290647,-1.120401,-1.473842,0.262269,0.478515,-0.995816,-1.330397,-0.799796,0.057534,0.382244,-0.353291,2.489086,0.678221,-0.386044,0.573989,DERMASON
1,1.079840,1.378508,0.895851,1.556096,-0.508400,-0.255766,1.086066,1.272269,0.780167,-0.710600,-1.167619,0.443399,-1.660960,-0.605992,0.411600,0.539056,BARBUNYA
2,-0.399247,-0.258129,-0.043884,-0.771747,1.228816,1.059278,-0.398674,-0.399408,-2.742996,-0.085355,-1.021913,-1.205649,0.962659,-0.687449,-1.185539,-0.461860,HOROZ
3,0.567520,0.853157,1.187270,0.239562,1.601924,1.222725,0.567135,0.756087,1.103026,-0.179103,-1.022033,-1.520070,-0.446775,-1.324419,-1.466396,-1.899307,CALI
4,-0.772396,-1.014998,-0.972606,-0.847312,-0.567204,-0.327418,-0.775575,-0.950097,0.487082,0.814363,1.117122,0.507116,1.075050,0.964806,0.477495,0.422180,DERMASON


In [21]:
test_data['Class'].unique()

array(['HOROZ', 'CALI', 'BARBUNYA', 'DERMASON', 'SIRA', 'BOMBAY', 'SEKER'],
      dtype=object)

**Before feeding the data into the following algorithms, try to perform PCA, varying the number of PCs, and check what changes**

## K-Nearest Neighbors Classification

Implement the KNN algorithm for classification.

In [22]:
from scipy.spatial.distance import euclidean, minkowski
from tqdm import tqdm

def distance(point_one, point_two, dist, p):
    if dist == 'euclidean':
        return euclidean(point_one, point_two)
    else:
        return minkowski(point_one, point_two, p=p)


def get_neighbors(train_set, test_point, label_col, n_neighbors, dist, p):
    # Calculate distances between the test point and all points in the training set
    dist = np.array([distance(train_point, test_point, dist, p) for train_point in train_set])
    # Get indices that would sort the distances in ascending order
    idx_dist = dist.argsort()
    # Order the training set and labels based on the sorted distances
    ordered_train = train_set[idx_dist, :]
    ordered_label = label_col[idx_dist]
    # Return the top n_neighbors neighbors and their labels
    return ordered_train[:n_neighbors], ordered_label[:n_neighbors]

def predict(train_set, test_point, labels, n_neighbors, dist, p):
    # Get the nearest neighbors and their labels
    neigh, neigh_label = get_neighbors(train_set, test_point, labels, n_neighbors, dist, p)
    # Count occurrences of each label among the neighbors
    values, counts = np.unique(neigh_label, return_counts=True)
    # Find the label with the highest count (majority class)
    idx = np.argmax(counts)
    # Return the predicted label
    return values[idx]

def evaluate(train_set, test_set, label, n_neighbors, dist='Euclidean', p=2):
    # Initialize counters for correct and incorrect predictions
    correct_predict = 0
    wrong_predict = 0
    # Extract labels and features from the training and test sets
    
    train_labels = train_set[label].values
    train_set = train_set.drop(label, axis=1)
    
    test_labels = test_set[label].values
    test_set = test_set.drop(label, axis=1)
    # Iterate through each row in the test dataset
    for index in tqdm(range(len(test_set.index))):
        # Predict the class label for the current test row
        result = predict(train_set.values, test_set.iloc[index].values, train_labels, n_neighbors, dist, p)
        # Check if the predicted value matches the actual value
        if result == test_labels[index]:
            # Increase the correct prediction count
            correct_predict += 1
        else:
            # Increase the incorrect prediction count
            wrong_predict += 1

    # Calculate and return the accuracy
    accuracy = correct_predict / (correct_predict + wrong_predict)
    return accuracy


In [23]:
knn_accuracy = evaluate(x_tr, x_te, "Class", n_neighbors=5)

100%|██████████| 3403/3403 [03:28<00:00, 16.34it/s]


In [83]:
knn_accuracy

0.9243976737745777

## Decision Trees with Numerical Features

Modify the implementation of decision trees to account for numerical input features.

In [45]:
# compute H(S)
def entropy(train_data, label, class_list):
    """
    Calculate the entropy of a dataset.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.

    Returns
    -------
    total_entr : float
        The entropy of the dataset.
    """
    # Get the total number of instances in the dataset
    total_row = train_data.shape[0]
    # Initialize the total entropy variable
    total_entr = 0

    # Iterate through each possible class in the label
    for c in class_list:
        # Count the number of points belonging to the current class
        total_class_count = train_data[train_data[label] == c].shape[0]

        # Check if there are instances of the class to avoid numerical errors
        if total_class_count > 0:
            # Calculate the entropy of the current class
            total_class_entr = - (total_class_count / total_row) * np.log2(total_class_count / total_row)
            # Add the entropy of the current class to the total entropy of the dataset
            total_entr += total_class_entr

    # Return the calculated total entropy of the dataset
    return total_entr

In [46]:
# compute H(S_j)
def feature_entropy(left_data, right_data, label, class_list):
    """
    Calculate the conditional entropy of a dataset split by a specific feature.

    Parameters
    ----------
    left_data : DataFrame
        Subset of the dataset where the feature has a specific value.
    right_data : DataFrame
        Subset of the dataset where the feature has another value.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.

    Returns
    -------
    ent : float
        The conditional entropy of the dataset split by the feature.
    """
    # Get the total number of points considered after the split
    row_count = left_data.shape[0] + right_data.shape[0]

    # Calculate the probabilities of the left and right subsets
    p_left = left_data.shape[0] / row_count
    p_right = right_data.shape[0] / row_count

    # Calculate the conditional entropy using the weighted average of entropies for left and right subsets
    ent = p_left * entropy(left_data, label, class_list) + p_right * entropy(right_data, label, class_list)

    # Return the calculated conditional entropy
    return ent

In [47]:
def split(feature_column, threshold):
    """
    Split the indices of data points based on a feature and a threshold.

    Parameters
    ----------
    feature_column : array-like
        The values of the feature for each data point.
    threshold : float
        The threshold value for splitting the data points.

    Returns
    -------
    left_rows : array-like
        Indices of data points where the feature value is less than or equal to the threshold.
    right_rows : array-like
        Indices of data points where the feature value is greater than the threshold.
    """
    # Find the indices of data points where the feature value is less than or equal to the threshold
    left_rows = np.argwhere(feature_column <= threshold).flatten()
    # Find the indices of data points where the feature value is greater than the threshold
    right_rows = np.argwhere(feature_column > threshold).flatten()

    # Return the indices for left and right subsets
    return left_rows, right_rows

In [48]:
def information_gain(data, feature_name, label, class_list, threshold):
    """
    Calculate the information gain after splitting the dataset based on a feature and a threshold.

    Parameters
    ----------
    data : DataFrame
        The dataset.
    feature_name : str
        The name of the feature for which information gain is calculated.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.
    threshold : float
        The threshold value for splitting the dataset.

    Returns
    -------
    feat_information_gain : float
        The information gain achieved by splitting the dataset based on the specified feature and threshold.
    """
    # Split the dataset into left and right subsets based on the feature and threshold
    left_rows, right_rows = split(data[feature_name].values, threshold)

    # Check if either subset is empty; if so, information gain is zero
    if len(left_rows) == 0 or len(right_rows) == 0:
        return 0

    # Calculate the entropy of the split dataset
    feat_entropy = feature_entropy(data.iloc[left_rows], data.iloc[right_rows], label, class_list)

    return feat_entropy


In [49]:
def get_split_thresholds(feature_column, n_thresholds):
    """
    Generate candidate split thresholds for a given feature column.

    Parameters
    ----------
    feature_column : array-like
        The values of the feature for each data point.
    n_thresholds : int
        The number of thresholds to generate.

    Returns
    -------
    thresholds : list of float
        List of candidate split thresholds for the feature column.
    """
    # Extract the values of the feature column
    feature_column = feature_column.values
    # Get the total number of data points
    n_data = len(feature_column)

    # Sort the feature column in ascending order
    sorted_column = np.sort(feature_column)

    # Check if there is more than one data point
    if len(feature_column) > 1:
        # Split the sorted feature column into n_thresholds + 1 partitions
        partitioned_array = np.array_split(sorted_column, n_thresholds + 1)

        # Calculate the midpoint between consecutive partitions as candidate thresholds
        thresholds = [(partitioned_array[i][-1] + partitioned_array[i + 1][0]) / 2 for i in range(len(partitioned_array) - 1)]
    else:
        # If there is only one data point, use it as the threshold
        thresholds = [feature_column[0]]

    # Return the list of candidate split thresholds
    return thresholds


In [50]:
def most_informative_feature(train_data, label, class_list, n_thresholds):
    """
    Find the most informative feature and its corresponding threshold for splitting the dataset.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.
    n_thresholds : int
        The number of thresholds to generate for each feature.

    Returns
    -------
    min_entropy_feature : str
        The name of the most informative feature.
    min_entropy_threshold : float
        The corresponding threshold for splitting the dataset based on the most informative feature.
    """
    # Get the list of features excluding the label
    feature_list = train_data.columns.drop(label)

    # Initialize variables to store the minimum entropy and corresponding feature and threshold
    min_entropy = float('inf')
    min_entropy_feature = None
    min_entropy_threshold = None

    # Iterate over each feature in the feature list
    for feature in feature_list:
        # Generate candidate split thresholds for the current feature
        thresholds = get_split_thresholds(train_data[feature], n_thresholds)

        # Iterate over each threshold
        for t in thresholds:
            # Calculate information gain for the current feature and threshold
            info_gain = information_gain(train_data, feature, label, class_list, t)

            # Check if the calculated information gain is less than the current minimum entropy
            if info_gain < min_entropy:
                # Update the minimum entropy and corresponding feature and threshold
                min_entropy = info_gain
                min_entropy_feature = feature
                min_entropy_threshold = t

    # Return the most informative feature and its corresponding threshold
    return min_entropy_feature, min_entropy_threshold

In [51]:
def is_leaf(train_data, label):
    """
    Check if a node in a decision tree is a leaf node.

    Parameters
    ----------
    train_data : DataFrame
        The dataset associated with the current node.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    bool
        True if the node is a leaf node (contains only one class), False otherwise.
    """
    # Get the unique classes in the current node
    classes_in_node = np.unique(train_data[label])

    # Check if there is only one class in the node
    if len(classes_in_node) == 1:
        # If there is only one class, the node is a leaf node
        return True
    else:
        # If there is more than one class, the node is not a leaf node
        return False

In [52]:
def leaf_class(train_data, label):
    """
    Determine the class of a leaf node in a decision tree.

    Parameters
    ----------
    train_data : DataFrame
        The dataset associated with the leaf node.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    leaf_class : str
        The class label assigned to the leaf node.
    """
    # Get the unique classes and their counts in the current leaf node
    class_list, count_class = np.unique(train_data[label], return_counts=True)

    # Find the index of the class with the highest count (most frequent class)
    idx = count_class.argmax()

    # Return the class label associated with the most frequent class in the leaf node
    return class_list[idx]

In [53]:
def make_tree(train_data, label, class_list, n_thresholds, cur_depth, min_samples, max_depth):
    """
    Recursively build a decision tree.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset associated with the current node.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.
    n_thresholds : int
        The number of thresholds to generate for each feature.
    cur_depth : int
        The current depth of the decision tree.
    min_samples : int
        The minimum number of samples required to split a node.
    max_depth : int
        The maximum depth of the decision tree.

    Returns
    -------
    tree : dict or str
        The constructed decision tree represented as a nested dictionary. If a leaf node, returns the class label.
    """
    # Check stopping conditions for creating a leaf node
    if is_leaf(train_data, label) or cur_depth >= max_depth or len(train_data) <= min_samples:
        return leaf_class(train_data, label)
    else:
        # Increment the current depth for the next level of recursion
        cur_depth += 1

        # Find the most informative feature and its corresponding threshold for splitting
        split_feature, split_threshold = most_informative_feature(train_data, label, class_list, n_thresholds)

        # Split the dataset into left and right subsets based on the feature and threshold
        left_rows, right_rows = split(train_data[split_feature].values, split_threshold)

        # Check if either subset is empty; if so, create a leaf node
        if len(left_rows) == 0 or len(right_rows) == 0:
            return leaf_class(train_data, label)
        else:
            # Build the subtree
            split_condition = "{} <= {}".format(split_feature, split_threshold)
            sub_tree = {split_condition: []}

            # Recursive calls for the left and right branches
            left_branch = make_tree(train_data.iloc[left_rows], label, class_list, n_thresholds, cur_depth, min_samples, max_depth)
            right_branch = make_tree(train_data.iloc[right_rows], label, class_list, n_thresholds, cur_depth, min_samples, max_depth)

            # Check if both branches result in the same leaf class; if so, make the subtree a leaf
            if left_branch == right_branch:
                sub_tree = left_branch
            else:
                # Grow the tree by adding left and right branches to the split condition
                sub_tree[split_condition].append(left_branch)
                sub_tree[split_condition].append(right_branch)

            return sub_tree

In [54]:
def id3(train_data_m, label, n_thresholds=1, min_samples=4, max_depth=6):
    """
    Build a decision tree using the ID3 algorithm.

    Parameters
    ----------
    train_data_m : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.
    n_thresholds : int, optional
        The number of thresholds to generate for each feature.
    min_samples : int, optional
        The minimum number of samples required to split a node.
    max_depth : int, optional
        The maximum depth of the decision tree.

    Returns
    -------
    tree : dict or str
        The constructed decision tree represented as a nested dictionary. If a leaf node, returns the class label.
    """
    # Create a copy of the training dataset
    train_data = train_data_m.copy()

    # Get the unique classes of the label
    class_list = train_data[label].unique()

    # Start the recursion by calling the make_tree function
    tree = make_tree(train_data, label, class_list, n_thresholds, 0, min_samples, max_depth)

    # Return the constructed decision tree
    return tree

In [55]:
t = id3(train_data, 'Class')
print(t)

{'MajorAxisLength <= -0.26850534253244707': [{'MinorAxisLength <= -0.48101943425512017': [{'Area <= -0.7284149650882628': ['DERMASON', {'Perimeter <= -0.7436366901340172': ['DERMASON', {'roundness <= 0.34196234239628087': [{'ShapeFactor1 <= 0.742560155983303': ['SIRA', 'DERMASON']}, 'DERMASON']}]}]}, {'AspectRation <= -1.0816890411884459': ['SEKER', {'ShapeFactor2 <= 0.30636007444314095': ['SIRA', {'ShapeFactor1 <= 0.14890193826388914': [{'Compactness <= 0.8530383347908611': ['SIRA', 'SEKER']}, 'DERMASON']}]}]}]}, {'ShapeFactor1 <= -0.4188395892843162': [{'ShapeFactor2 <= -0.970259723740702': [{'Perimeter <= 1.2423181848501317': ['CALI', {'ShapeFactor1 <= -2.601011987728354': ['BOMBAY', 'CALI']}]}, {'roundness <= -0.655896778746935': ['BARBUNYA', {'Compactness <= -0.19114716081240862': ['CALI', {'roundness <= -0.07128514945601658': ['BARBUNYA', 'CALI']}]}]}]}, {'Compactness <= -1.1840258480155355': ['HOROZ', {'roundness <= 0.019868264666345184': [{'roundness <= -0.33450228559615847': [

In [56]:
def predict(test_point, tree):
    """
    Predict the class label for a given test point using a decision tree.

    Parameters
    ----------
    test_point : Series
        The test point for which the class label is predicted.
    tree : dict or str
        The decision tree used for prediction.

    Returns
    -------
    prediction : str
        The predicted class label for the test point.
    """
    # Base case: if the tree is a leaf node (a class label)
    if not isinstance(tree, dict):
        return tree

    # Recursive case: traverse the tree based on feature values
    question = list(tree.keys())[0]
    attribute, value = question.split(" <= ")

    # Check the condition and follow the appropriate branch
    if test_point[attribute] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # Recursive call on the selected branch
    return predict(test_point, answer)


def evaluate(tree, test_data, label):
    """
    Evaluate the accuracy of a decision tree on a test dataset.

    Parameters
    ----------
    tree : dict or str
        The decision tree to be evaluated.
    test_data : DataFrame
        The test dataset.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    accuracy : float
        The accuracy of the decision tree on the test dataset.
    """
    correct_predict = 0
    wrong_predict = 0

    # Iterate over each row in the test dataset
    for index in tqdm(range(len(test_data.index))):
        # Predict the class label for the current test point
        result = predict(test_data.iloc[index], tree)

        # Check if the predicted value matches the expected value
        if result == test_data[label].iloc[index]:
            correct_predict += 1  # Increase correct count
        else:
            wrong_predict += 1  # Increase incorrect count

    # Calculate and return the accuracy
    accuracy = correct_predict / (correct_predict + wrong_predict)
    return accuracy

In [57]:
evaluate(t, test_data, 'Class')

100%|██████████| 3611/3611 [00:00<00:00, 23496.46it/s]


0.8773193021323733

## Gaussian Naive Bayes
Modify the implementation of naive Bayes to accout for numerical input features. The likelihood of each class ($p(data|class)$) is assumed to be a Gaussian $\frac{1}{\sqrt(\sigma^2 2 \pi)} \exp (-\frac{1}{2} \frac{(x-\mu)^2}{\sigma^2})$, where $\mu, \sigma^2$ are the mean and the variance for each class;

In [58]:
def prior(train_data, label):
    """
    Calculate the log prior probabilities for each class in the dataset.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    priors : array-like
        The log prior probabilities for each class.
    """
    # Calculate the prior probabilities for each class
    priors = train_data.groupby(by=label).apply(lambda x: len(x) / len(train_data))

    # Return the log of the prior probabilities as an array
    return np.log(priors).values


def mean_variance(train_data, label):
    """
    Calculate the mean and variance for each feature in the dataset, grouped by class.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    mean : array-like
        The mean values for each feature and class.
    variance : array-like
        The variance values for each feature and class.
    """
    # Calculate the mean values for each feature and class
    mean = train_data.groupby(by=label).apply(lambda x: x.mean(axis=0))

    # Calculate the variance values for each feature and class
    variance = train_data.groupby(by=label).apply(lambda x: x.var(axis=0))

    # Return the mean and variance as arrays
    return (mean.values, variance.values + 1e-9)


def gaussian_density(mean, variance, point):
    """
    Calculate the Gaussian probability density for a given point.

    Parameters
    ----------
    mean : array-like
        The mean values for each feature and class.
    variance : array-like
        The variance values for each feature and class.
    point : array-like
        The values of the features for a given point.

    Returns
    -------
    density : array-like
        The Gaussian probability density for the given point.
    """
    # Calculate the Gaussian probability density for each feature
    d = (1 / np.sqrt(2*np.pi*variance)) * np.exp((-(point - mean)**2) / (2*variance))

    # Return the density as an array
    return d


def train_gaussian_naive_bayes(train_data, label):
    """
    Train a Gaussian Naive Bayes classifier.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    model : dict
        A dictionary containing the parameters of the trained Gaussian Naive Bayes model.
    """
    # Calculate the mean and variance for each feature and class
    mean, variance = mean_variance(train_data, label)

    # Calculate the log prior probabilities for each class
    priors = prior(train_data, label)

    # Get unique class labels and their count
    unique_labels = train_data[label].unique()
    n_labels = len(unique_labels)

    # Construct and return the Gaussian Naive Bayes model
    return {'n_labels': n_labels, 'unique_labels': unique_labels, 'n_classes': n_labels, 'mean': mean,
            'variance': variance, 'prior': priors}


In [59]:
gaus_bayes = train_gaussian_naive_bayes(train_data, 'Class')

C:\Users\pitom\AppData\Local\Temp\ipykernel_1680\1237244830.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  priors = train_data.groupby(by=label).apply(lambda x: len(x) / len(train_data))


In [60]:
print(gaus_bayes['mean'].shape)

(7, 16)


In [61]:
def posterior(point, mean, variance, class_list, n_classes, n_feat):
    """
    Calculate the log posterior probabilities for each class given a data point.

    Parameters
    ----------
    point : array-like
        The values of the features for a given data point.
    mean : array-like
        The mean values for each feature and class.
    variance : array-like
        The variance values for each feature and class.
    class_list : array-like
        The unique class labels.
    n_classes : int
        The number of classes.
    n_feat : int
        The number of features.

    Returns
    -------
    posteriors : array-like
        The log posterior probabilities for each class.
    """
    posteriors = []
    for i in range(n_classes):
        posterior = 0
        for j in range(n_feat):
            posterior += np.log(gaussian_density(mean[i][j], variance[i][j], point[j]))
        posteriors.append(posterior)
    return posteriors


def predict(test_data, label, gaus_bayes):
    """
    Predict the class labels for a given test dataset using a trained Gaussian Naive Bayes model.

    Parameters
    ----------
    test_data : DataFrame
        The test dataset.
    label : str
        The name of the column representing the class labels.
    gaus_bayes : dict
        A dictionary containing the parameters of the trained Gaussian Naive Bayes model.

    Returns
    -------
    predictions : array-like
        The predicted class labels for the test dataset.
    """
    predictions = []
    n_feat = len(test_data.columns) - 1
    for i in range(len(test_data)):
        pr = gaus_bayes['prior']
        post = posterior(test_data.iloc[i, :-1], gaus_bayes['mean'], gaus_bayes['variance'],
                         gaus_bayes['unique_labels'], gaus_bayes['n_classes'], n_feat)
        prob = pr + post
        max_prob_class_idx = np.argmax(prob)
        predictions.append(gaus_bayes['unique_labels'][max_prob_class_idx])
    return predictions


def evaluate(test_data, label, gaus_bayes):
    """
    Evaluate the accuracy of a Gaussian Naive Bayes model on a test dataset.

    Parameters
    ----------
    test_data : DataFrame
        The test dataset.
    label : str
        The name of the column representing the class labels.
    gaus_bayes : dict
        A dictionary containing the parameters of the trained Gaussian Naive Bayes model.

    Returns
    -------
    accuracy : float
        The accuracy of the Gaussian Naive Bayes model on the test dataset.
    """
    gaus_pred = predict(test_data, label, gaus_bayes)
    correct_predict = 0
    wrong_predict = 0
    for index in tqdm(range(len(test_data.index))):
        if gaus_pred[index] == test_data[label].iloc[index]:
            correct_predict += 1
        else:
            wrong_predict += 1
    accuracy = correct_predict / (correct_predict + wrong_predict)
    return accuracy

In [62]:
evaluate(test_data, 'Class', gaus_bayes)

C:\Users\pitom\AppData\Local\Temp\ipykernel_1680\116866810.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  posterior += np.log(gaussian_density(mean[i][j], variance[i][j], point[j]))
C:\Users\pitom\AppData\Local\Temp\ipykernel_1680\116866810.py:29: RuntimeWarning: divide by zero encountered in log
  posterior += np.log(gaussian_density(mean[i][j], variance[i][j], point[j]))
100%|██████████| 3611/3611 [00:00<00:00, 199054.14it/s]


0.003323179174743838

In [63]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

cols = list(set(train_data.columns) - {'Class'})

nb.fit(train_data[cols], train_data['Class'])

pred = nb.predict(test_data[cols])

In [51]:
print((pred == label_test.values).sum()/len(pred))

0.903073940736638
